In [4]:
import numpy as np
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
np.random.seed(2018)
from gensim import models

### DATA

In [5]:
news_all= pd.read_csv("test_news_translated.csv")

###  DATA CLEANING

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield (gensim.utils.simple_preprocess(str(sentence),deacc=True))
        
stopwords_verbs = ['say',"ecuadorian","jair","mauricio","argentinian","bueno","summari","paraguayan","uruguayan","franci","lionel","francisco","chang","hondura","honduran","julian","nicaraguan","lenin","best","confirm","tell","stori","month","year","president","colombian","ecuador","presid",'lenin','get', 'go','know', 'may', 'need', 'like', 'make', 'see', 'want', 'come', 'take', 'use', 'would', 'can']
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if (len(word)>3) and (word not in stopwords_verbs) ] for doc in texts]


def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def crearMatriz(dic,doc,corpus):
    matriz= np.zeros((len(doc),len(dic)))
    for doc in range(len(doc)):
        for word in range(len(corpus[doc])):
            matriz[doc][(corpus[doc][word][0])]+= corpus[doc][word][1]  
    return pd.DataFrame(matriz)

### TOPIC MODELING

In [ ]:
def dictionary(data, filter_extreme=True,below=5, above=0.5):
    id2word = gensim.corpora.Dictionary(data_words_nostops)# sin filtroooooo
    if filter_extreme:
        id2word.filter_extremes(no_below= below, no_above= above, keep_n=100000)
        return id2word
    return id2word
        
    
def corpus(data):
    corpus = [id2word.doc2bow(data) for text in data] # Bag of words
    return corpus



def top_of_words(data,id2word,corpus,save=True):
        bow=crearMatriz(dic=id2word,doc= data_words_nostops , corpus=corpus)
        lista_palabras=[]
        for i in range(len(id2word)):
            lista_palabras.append(id2word[i])
        bow.columns = lista_palabras
        top_palabras= bow.sum(axis=0).sort_values(ascending=False) 
        words=pd.DataFrame(top_palabras.head(10)).reset_index()
        words.columns= ["words", "count"]
        words["country"]=pd.DataFrame([country]*10)
        words["ano"]=pd.DataFrame([ano]*10)
        listaTops.append(words)
        if save:
            words.to_csv(country+"_"+str(ano)+".csv")
            
            
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    sent_topics_df = pd.DataFrame()
    
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num)+1, round(prop_topic,4), topic_keywords, ano]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords', "year"]
    sent_topics_df["country"]=pd.DataFrame([country]*len(noticias))
    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)                



def topic_modeling(news_all, country, year):  
        news= news_all[(news_all["country"]==country)&(news_all["year"]==ano)]
        #data= news["tokens-title-stopwords-stemming"].values
        doc_words= list(sent_to_words(news["tokens-title-stopwords-stemming"].values))
        data_words_nostops= remove_stopwords(doc_words)
        id2word= dictionary(data_words_nostops, filter_extreme=True,below=5, above=0.5)
        corpus= corpus(data_words_nostops)
        #model 
        tfidf= models.TfidfModel(corpus)
        corpus_tfidf= tfidf[corpus]
        lda_model_TFIDF = gensim.models.LdaMulticore(corpus=corpus_tfidf,id2word=id2word,num_topics=5,random_state=100,chunksize=50,passes=10,per_word_topics=True)
        return lda_model_TFIDF

In [ ]:
all_data=[]
listaTops=[]
for pais in paises:
    for year in years:
        topic_modeling(news_all, country, year)
        lista_news= [new for new in news.title]
        df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model_TFIDF, corpus=corpus_tfidf,texts=lista_news)
        all_data.append(df_topic_sents_keywords)
result = pd.concat(all_data)

In [19]:
result

,Dominant_Topic,Perc_Contribution,Topic_Keywords,year,country,0
0,5.0,0.6651,"final, abort, river, boca, iceland, return, do...",2018.0,argentina,Argentina: el Senado rechaza la nueva ley de a...
1,3.0,0.3642,"messi, footbal, match, russia, beat, peso, sel...",2018.0,argentina,Rusia 2018: Argentina clasifica a octavos de f...
2,1.0,0.6942,"nigeria, franc, spain, face, market, emerg, ge...",2018.0,argentina,Eliminada Argentina de Rusia 2018 al caer 4-3 ...
3,2.0,0.4514,"macri, play, croatia, coach, goal, talk, suppo...",2018.0,argentina,Rusia 2018: Croacia golea a una decepcionante ...
4,5.0,0.4946,"final, abort, river, boca, iceland, return, do...",2018.0,argentina,ARA San Juan: por qué es difícil rescatar el s...
...,...,...,...,...,...,...
604,5.0,0.6000,"sanction, kill, border, offici, drone, worst, ...",2018.0,venezuela,UN 'running out of cash' and facing urgent cut...
605,1.0,0.2000,"nicola, start, earthquak, accus, russia, jose,...",2018.0,venezuela,Inside the 14 December edition of the Guardian...
606,3.0,0.6000,"crisi, currenc, price, product, spain, govern,...",2018.0,venezuela,Oil prices could spiral higher in the coming m...
607,2.0,0.6000,"vinotinto, photo, hunger, hyperinfl, flee, mig...",2018.0,venezuela,World Press Photo Contest 2018 – the nominees ...


In [20]:
#words.to_excel('topWords_todosPaises2.xlsx')
result.to_excel("topicos_todosPaisesFILTRO.xlsx")